### 依赖安装
安装一些必须的依赖包。

In [1]:
!mkdir /home/aistudio/external-libraries
# !pip3 install -U paddlepaddle-gpu
!pip3 install -U paddlenlp
!pip3 install trustai==0.1.5 -t /home/aistudio/external-libraries
# !pip install trustai -t /home/aistudio/external-libraries

mkdir: 无法创建目录"/home/aistudio/external-libraries": 文件已存在
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 852.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 711.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 7.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 5.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

### 数据准备
#### 1）模型训练数据
我们推荐使用LCQMC数据集训练中文相似度计算模型。Paddlenlp框架会自动下载及缓存训练数据集，默认缓存存储路径为"~/.paddlenlp/datasets"。如需修改训练数据，请参考『初始化工作』中DATASET_NAME的修改。
#### 2）下载预训练模型
基线使用了ERNIE-3.0-base预训练模型。Paddlenlp框架自动缓存模型文件，默认缓存存储路径为"~/.paddlenlp/models"。如需修改依赖的预训练模型，请在『初始化工作』中修改MODEL_NAME。

### 初始化工作
初始化工作包括了模型选择及加载、训练数据集选择、模型存储路径设定、抽取证据的长度占原文本长度的比例设定等。可按需更改。

In [2]:
import sys
sys.path.insert(0, '/home/aistudio/work/trustai-pkg-paddle')
sys.path.insert(0, '/home/aistudio/external-libraries')
import json
import numpy as np
import paddle
import paddlenlp
from paddlenlp.transformers import ErnieForSequenceClassification, ErnieTokenizer

# Select pre-trained model
MODEL_NAME = "ernie-3.0-base-zh" # choose from ["ernie-1.0", "ernie-1.0-base-zh", "ernie-1.0-large-zh-cw", "ernie-2.0-base-zh", "ernie-2.0-large-zh", "ernie-3.0-xbase-zh", "ernie-3.0-base-zh", "ernie-3.0-medium-zh", "ernie-3.0-mini-zh", "ernie-3.0-micro-zh", "ernie-3.0-nano-zh"]
# Select dataset for model training
DATASET_NAME = 'lcqmc'
# Set the path to save the trained model
MODEL_SAVE_PATH = f'./save_model/{DATASET_NAME}-{MODEL_NAME}'
# Set the rationale length ratio which determines the length of the extracted rationales.
RATIONALE_RATIO = 0.705 # 0.705 for Chinese dataset, 0.508 for English dataset

/home/aistudio/external-libraries/_distutils_hack/__init__.py:19: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  "Distutils was imported before Setuptools, but importing Setuptools "
/home/aistudio/external-libraries/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-pack

In [3]:
# Init model and tokenizer
model = ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

[2023-01-28 14:06:25,821] [    INFO] - Model config ErnieConfig {
  "attention_probs_dropout_prob": 0.1,
  "enable_recompute": false,
  "fuse": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "ernie",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "paddlenlp_version": null,
  "pool_act": "tanh",
  "task_id": 0,
  "task_type_vocab_size": 3,
  "type_vocab_size": 4,
  "use_task_id": true,
  "vocab_size": 40000
}

[2023-01-28 14:06:25,825] [    INFO] - Configuration saved in /home/aistudio/.paddlenlp/models/ernie-3.0-base-zh/config.json
[2023-01-28 14:06:25,830] [    INFO] - Downloading ernie_3.0_base_zh.pdparams from https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_base_zh.pdparams
100%|██████████| 452M/452M [00:12<00:00, 37.6MB/s] 
W0128 14:06:40.580371   207 gpu_re

### 模型训练
这里以ERNIE-3.0为例训练一个文本相似度模型。

In [4]:
from paddlenlp.datasets import load_dataset
# Load dataset
train_ds, dev_ds, test_ds = load_dataset(DATASET_NAME, splits=["train", "dev", "test"])

100%|██████████| 6827/6827 [00:01<00:00, 4993.48it/s]


In [5]:
from importlib import reload
from trustai_pkg import prepare
reload(prepare)

<module 'trustai_pkg.prepare' from '/home/aistudio/work/trustai-pkg-paddle/trustai_pkg/prepare.py'>

In [6]:
train_ds[0], dev_ds[0], test_ds[0]

({'query': '喜欢打篮球的男生喜欢什么样的女生', 'title': '爱打篮球的男生喜欢什么样的女生', 'label': 1},
 {'query': '开初婚未育证明怎么弄？', 'title': '初婚未育情况证明怎么开？', 'label': 1},
 {'query': '谁有狂三这张高清的', 'title': '这张高清图，谁有', 'label': ''})

In [7]:
# 原始字段需要移除
remove_columns = ['source', 'target', 'id']
# 文本的最大长度
max_source_length = 128
max_seq_length = 128
# 摘要的最大长度
max_target_length = 64


model_set = prepare.ModelSet(tokenizer, model)
prepare_config = prepare.SimilarityPrepareConfig(text_column=remove_columns[0], target_column=remove_columns[1],
    max_seq_length=max_source_length, max_source_length=max_source_length, max_target_length=max_target_length, min_target_length=0, batch_size=12)

prepare_tool = prepare.SimilarityPrepare(model_set, prepare_config)

In [8]:
train_ds, dev_ds, test_ds = prepare_tool.get_dataset(train_ds, dev_ds, test_ds)

[2023-01-19 20:29:01,690] [   DEBUG] - tt:<class 'paddlenlp.datasets.dataset.MapDataset'>
[2023-01-19 20:29:01,692] [   DEBUG] - t:<class 'paddlenlp.datasets.dataset.MapDataset'>


In [9]:
train_dl, dev_dl, test_dl = prepare_tool.get_dataloader(train_ds, dev_ds, test_ds)

In [10]:
# # 输出训练集的前 3 条样本
# for idx, example in enumerate(dev_dl):
#     if idx < 3:
#         print(example) 

# 训练模型

In [21]:
from trustai_pkg import trainer
reload(trainer)

<module 'trustai_pkg.trainer' from '/home/aistudio/work/trustai-pkg-paddle/trustai_pkg/trainer.py'>

In [22]:
train_config = trainer.SimilarityTrainerConfig(
    # 学习率预热比例
    warmup = 0.02,
    # 学习率
    learning_rate = 5e-5,
    # 训练轮次
    num_epochs = 3,
    # AdamW优化器参数epsilon
    adam_epsilon = 1e-6,
    # AdamW优化器参数weight_decay
    weight_decay=0.01,
    # 训练中，每个log_steps打印一次日志
    log_steps = 100,
    # 训练中，每隔eval_steps进行一次模型评估
    eval_steps = 1000,
    # 摘要的最小长度
    min_target_length = 0,
    # 训练模型保存路径
    output_dir = 'checkpoints',
    # 解码beam size
    num_beams = 4,
    log_dir = 'visualdl_log_dir'
)

trainer = trainer.SimilarityTrainer(train_dl, dev_dl, model_set, prepare_config, train_config)

In [23]:
# for step, batch in enumerate(train_dl):
#     if step > 0:
#         break
#     print(batch)

In [ ]:
trainer.train()

[2023-01-19 21:05:55,513] [    INFO] - global step 100/59694, epoch: 0, batch: 99, rank_id: 0, loss: 0.700589, lr: 0.0000041911, speed: 0.0654 step/s


### 重要度分数获取
该步为输入中每个词赋一个重要度分数，表示该词对预测的影响度。重要度分数获取共分三步。
#### 1）加载模型和评测数据集
更改模型以及评估数据的存储路径（MODEL_PATH和DATA_PATH），完成模型和评测数据集的加载。赛段一数据量为1712条，赛段二数据量为4167条，请确认评测数据集完整。

In [4]:
from utils import load_data

# Correct MODEL_PATH and DATA_PATH before executing
MODEL_PATH = MODEL_SAVE_PATH + '/model_state.pdparams'
# MODEL_PATH = os.path.join(train_config.output_dir, 'model_state.pdparam')
DATA_PATH = '/home/aistudio/similarity_ch/similarity_ch'

# Load the trained parameters
state_dict = paddle.load(MODEL_PATH)
model.set_dict(state_dict)

# Load test data
data = load_data(DATA_PATH)
print("Num of data:", len(data))

Num of data: 4167



#### 2）数据预处理

a) 输入格式化：将输入的两个文本组织成模型预测所需格式，如对于Ernie3.0-base模型，其输入形式为[CLS]query[SEP]title[SEP]

b) 分词位置索引：计算每个分词结果对应的原文位置索引，这里的分词包括模型分词和标准分词

In [5]:
from trustai.interpretation import get_word_offset

# Add CLS and SEP tags to both original text and standard splited tokens
contexts = []
standard_split = []
for idx in data:
    example = data[idx]
    contexts.append("[CLS]" + example['query'] + "[SEP]" + example['title'] + "[SEP]")
    standard_split.append(["[CLS]"] + example['text_q_seg'] + ["[SEP]"] + example['text_t_seg'] + ["[SEP]"])

# Get the offset map of tokenized tokens and standard splited tokens
ori_offset_maps = []
standard_split_offset_maps = []
for i in range(len(contexts)):
    ori_offset_maps.append(tokenizer.get_offset_mapping(contexts[i]))
    standard_split_offset_maps.append(get_word_offset(contexts[i], standard_split[i]))

#### 3）重要度分数获取
我们提供attention，IG，LIME等三种解释方法，可根据实际实验结果选取最有效的一种方法。

##### a） Attention-based Interpreter

In [6]:
from trustai.interpretation.token_level.common import attention_predict_fn_on_paddlenlp
from trustai.interpretation.token_level import AttentionInterpreter
from utils import create_dataloader_from_scratch
# Hyperparameters
BATCH_SIZE = 8

# Init an attention interpreter and get the importance scores
att = AttentionInterpreter(model, device="gpu", predict_fn=attention_predict_fn_on_paddlenlp)

# Use attention interpreter to get the importance scores for all data
interp_results = None
for batch in create_dataloader_from_scratch(list(data.values()), tokenizer, BATCH_SIZE):
    if interp_results:
        interp_results += att(batch)
    else:
        interp_results = att(batch)

# Align the results back to the standard splited tokens so that it can be evaluated correctly later
align_res = att.alignment(interp_results, contexts, standard_split, standard_split_offset_maps, ori_offset_maps, special_tokens=["[CLS]", '[SEP]'])

##### b）IG-based Interpreter

In [8]:
from trustai.interpretation.token_level import IntGradInterpreter
from utils import create_dataloader_from_scratch
# Hyperparameters
IG_STEP = 100
BATCH_SIZE = 8

# Init an IG interpreter
ig = IntGradInterpreter(model, device="gpu")

# Use IG interpreter to get the importance scores for all data
interp_results = None
for batch in create_dataloader_from_scratch(list(data.values()), tokenizer, BATCH_SIZE):
    if interp_results:
        interp_results += ig(batch, steps=IG_STEP)
    else:
        interp_results = ig(batch, steps=IG_STEP)

# Align the results back to the standard splited tokens so that it can be evaluated correctly later
align_res = ig.alignment(interp_results, contexts, standard_split, standard_split_offset_maps, ori_offset_maps, special_tokens=["[CLS]", '[SEP]'])

##### c）LIME Interpreter

In [9]:

from trustai.interpretation.token_level import LIMEInterpreter
from utils import create_dataloader_from_scratch
# Hyperparameters
LIME_SAMPLES = 1000
BATCH_SIZE = 8

# Init an LIME interpreter
lime = LIMEInterpreter(model, device="gpu",
    unk_id=tokenizer.convert_tokens_to_ids('[UNK]'),
    pad_id=tokenizer.convert_tokens_to_ids('[PAD]'))

# Use LIME interpreter to get the importance scores for all data
interp_results = None
for batch in create_dataloader_from_scratch(list(data.values()), tokenizer, BATCH_SIZE):
    if interp_results:
        interp_results += lime(batch, num_samples=LIME_SAMPLES)
    else:
        interp_results = lime(batch, num_samples=LIME_SAMPLES)
    
# Align the results back to the standard splited tokens so that it can be evaluated correctly later
align_res = lime.alignment(interp_results, contexts, standard_split, standard_split_offset_maps, ori_offset_maps, special_tokens=["[CLS]", '[SEP]'])

### 生成用于评估的数据
评估文件格式要求是4列数据：编号\t预测标签\t证据1\t证据2，我们提供了脚本将模型输出结果转成评估所需格式。

In [10]:
import math

# Re-sort the token index according to their importance scores
def resort(index_array, importance_score):
    res = sorted([[idx, importance_score[idx]] for idx in index_array], key=lambda x:x[1], reverse=True)
    res = [n[0] for n in res]
    return res

# Post-prepare the result data so that it can be used for the evaluation directly
def prepare_eval_data(data, results, paddle_model):
    res = {}
    for data_id, inter_res in zip(data, results):
        # Split importance score vectors for query and title from inter_res.word_attributions
        query_importance_score = np.array(inter_res.word_attributions[1:len(data[data_id]['text_q_seg'])+1])
        title_importance_score = np.array(inter_res.word_attributions[len(data[data_id]['text_q_seg'])+2:-1])
        # Extract topK importance scores
        query_topk = math.ceil(len(data[data_id]['text_q_seg'])*RATIONALE_RATIO)
        title_topk = math.ceil(len(data[data_id]['text_t_seg'])*RATIONALE_RATIO)
        
        eval_data = {}        
        eval_data['id'] = data_id
        eval_data['pred_label'] = inter_res.pred_label
        # Find the token index of the topK importance scores
        eval_data['rationale_q'] = np.argpartition(query_importance_score, -query_topk)[-query_topk:]
        eval_data['rationale_t'] = np.argpartition(title_importance_score, -title_topk)[-title_topk:]
        # Re-sort the token index according to their importance scores
        eval_data['rationale_q'] = resort(eval_data['rationale_q'], query_importance_score)
        eval_data['rationale_t'] = resort(eval_data['rationale_t'], title_importance_score)

        res[data_id] = eval_data
    return res

# Generate results for evaluation
predicts = prepare_eval_data(data, align_res, model)
out_file = open('./sim_rationale.txt', 'w')
for key in predicts:
    out_file.write(str(predicts[key]['id'])+'\t'+ str(predicts[key]['pred_label'])+'\t')
    for idx in predicts[key]['rationale_q'][:-1]:
        out_file.write(str(idx)+',')
    out_file.write(str(predicts[key]['rationale_q'][-1])+'\t')

    for idx in predicts[key]['rationale_t'][:-1]:
        out_file.write(str(idx)+',')
    out_file.write(str(predicts[key]['rationale_t'][-1])+'\n')
out_file.close()